In [1]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

from joblib import Parallel, delayed
import time

from skimage.transform import rescale

Setting environment for Gordon


In [2]:
def add_annotation_visualization(stack, sec, bg='original', downscale_factor=8, users=['yuncong', 'autoAnnotate'],
                                show_labels=True):
    
    dm.set_stack(stack)
    dm.set_slice(sec)
    
    if 'yuncong' in users:    
        ret = dm.load_proposal_review_result(stack=stack, section=sec, 
                                             username='yuncong', timestamp='latest', suffix='consolidated')
        if ret is not None:
            user, ts, _, annotations = ret
    
    if 'autoAnnotate' in users:        
        ret_autoAnno = dm.load_proposal_review_result(stack=stack, section=sec, 
                                                      username='autoAnnotate', timestamp='latest', suffix='consolidated')
        if ret_autoAnno is not None:
            user2, ts2, _, annotations_autoAnno = ret_autoAnno        

#     ret_autoAnno = None

    if ret is None and ret_autoAnno is None:
        return
        
    if bg == 'original':
        img = imread(dm._get_image_filepath(stack=stack, section=sec, version='rgb-jpg'))
        viz = img[::downscale_factor, ::downscale_factor].copy()
#     viz = img_as_ubyte(rescale(img, 1./downscale_factor))
    else:
        viz = bg.copy()
    
    label_texts = set([])
    
    if ret is not None and len(annotations) > 0:
            
        for ann in annotations:

            vertices = np.array(ann['vertices']) / downscale_factor

        #     for x,y in vertices:
        #         cv2.circle(viz, (int(x), int(y)), 5, (0,0,255), -1)
            cv2.polylines(viz, [vertices.astype(np.int)], True, (0,0,255), 5)
            
            if show_labels:
                lx, ly = np.array(ann['labelPos']) / downscale_factor
                cv2.putText(viz, ann['label'], (int(lx)-10, int(ly)+10), 
                            cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 2)

            label_texts.add(ann['label'])
            
            
    if ret_autoAnno is not None and len(annotations_autoAnno) > 0:
        
        for ann in annotations_autoAnno:
            
#             vertices_ref = np.array(ann['vertices']) / downscale_factor
            
            vertices = np.array(ann['vertices']) / downscale_factor

        #     for x,y in vertices:
        #         cv2.circle(viz, (int(x), int(y)), 5, (255,0,0), -1)
            cv2.polylines(viz, [vertices.astype(np.int)], True, (255,0,0), 2)
            
            if show_labels:
            
                if ann['label'] not in label_texts:            
                    lx, ly = np.array(ann['labelPos']) / downscale_factor
                    cv2.putText(viz, ann['label'], (int(lx)-10, int(ly)+10), 
                                cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 2)


    if ret is not None and ret_autoAnno is not None:
        output_fn = os.path.join(labelingViz_dir, 
                     '%(stack)s_%(sec)04d_%(user)s_%(ts)s_%(user2)s_%(ts2)s_viz.jpg'% \
                                 {'stack':stack, 'sec':sec, 'user': user, 'ts': ts, 'user2':user2, 'ts2':ts2})
    elif ret is not None:
        output_fn = os.path.join(labelingViz_dir, 
                     '%(stack)s_%(sec)04d_%(user)s_%(ts)s_viz.jpg'% \
                                 {'stack':stack, 'sec':sec, 'user': user, 'ts': ts})            
    else:
        output_fn = os.path.join(labelingViz_dir, 
             '%(stack)s_%(sec)04d_%(user2)s_%(ts2)s_viz.jpg'% \
                         {'stack':stack, 'sec':sec, 'user2': user2, 'ts2': ts2})
            
    cv2.imwrite(output_fn, viz[..., ::-1])
    
    return viz

In [3]:
dm = DataManager()

In [5]:
for stack in ['MD589']:

    labelingViz_dir = os.path.join(labelingViz_root, stack)
    os.system('rm -r %s' % labelingViz_dir)
    create_if_not_exists(labelingViz_dir)
    
    # parallel

    first_bs_sec, last_bs_sec = section_range_lookup[stack]

    import time
    t = time.time()

    _ = Parallel(n_jobs=8)(delayed(add_annotation_visualization)(stack, sec) 
                           for sec in range(first_bs_sec, last_bs_sec+1))

    # simultaneous write seems fine
    sys.stderr.write('generate visualization: %.2f seconds\n' % (time.time() - t)) # ~ 60s/150 sections; ~200s/stack

generate visualization: 177.42 seconds
username yuncong does not have any annotations for current section 96 
username yuncong does not have any annotations for current section 97 
username yuncong does not have any annotations for current section 94 
username yuncong does not have any annotations for current section 99 
username yuncong does not have any annotations for current section 98 
username yuncong does not have any annotations for current section 100 
username yuncong does not have any annotations for current section 95 
username yuncong does not have any annotations for current section 93 
username autoAnnotate does not have any annotations for current section 96 
username autoAnnotate does not have any annotations for current section 97 
username autoAnnotate does not have any annotations for current section 94 
username autoAnnotate does not have any annotations for current section 99 
username autoAnnotate does not have any annotations for current section 98 
username aut

In [ ]:
downscale_factor = 8

In [ ]:
########## serial version ##########

first_bs_sec, last_bs_sec = section_range_lookup[stack]

import time

for sec in range(first_bs_sec, last_bs_sec+1):
    t = time.time()

    visualize_annotation(sec)

    sys.stderr.write('generate visualization: %.2f seconds\n' % (time.time() - t))

In [ ]:
########## serial version ##########

first_bs_sec, last_bs_sec = section_range_lookup[stack]

for sec in range(first_bs_sec, last_bs_sec+1):
# for sec in range(181, 262+1):
# for sec in [135]:

    dm.set_slice(sec)
    
    ret = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')
    if ret is not None:
        user, ts, _, annotations = ret
        
#     ret_autoAnno = dm.load_proposal_review_result('autoAnnotate', 'latest', 'consolidated')
#     if ret_autoAnno is not None:
#         user2, ts2, _, annotations_autoAnno = ret_autoAnno        
    ret_autoAnno = None

    if ret is None and ret_autoAnno is None:
        continue
    
    print '\n'
    print sec
    
    img = imread(os.path.join(os.environ['DATA_DIR'], '%(stack)s_lossless_aligned_cropped_downscaled', 
                  '%(stack)s_%(sec)04d_lossless_aligned_cropped_downscaled.jpg') % {'stack': stack, 'sec': sec})

#     viz = img_as_ubyte(rescale(img, 1./downscale_factor))
    viz = img[::downscale_factor, ::downscale_factor].copy()
    
    label_texts = set([])
    
    if ret is not None and len(annotations) > 0:
            
        for ann in annotations:

            vertices = np.array(ann['vertices']) / downscale_factor

        #     for x,y in vertices:
        #         cv2.circle(viz, (int(x), int(y)), 5, (0,0,255), -1)
            cv2.polylines(viz, [vertices.astype(np.int)], True, (0,0,255), 5)

            if show_labels:
                lx, ly = np.array(ann['labelPos']) / downscale_factor
                cv2.putText(viz, ann['label'], (int(lx)-10, int(ly)+10), 
                            cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 2)

            label_texts.add(ann['label'])
            
            
    if ret_autoAnno is not None and len(annotations_autoAnno) > 0:
        
        for ann in annotations_autoAnno:
            
            
#             vertices_ref = np.array(ann['vertices']) / downscale_factor
            
            vertices = np.array(ann['vertices']) / downscale_factor

        #     for x,y in vertices:
        #         cv2.circle(viz, (int(x), int(y)), 5, (255,0,0), -1)
            cv2.polylines(viz, [vertices.astype(np.int)], True, (255,0,0), 2)
            
            print ann['label']

            if show_labels:
            
                if ann['label'] not in label_texts:            
                    lx, ly = np.array(ann['labelPos']) / downscale_factor
                    cv2.putText(viz, ann['label'], (int(lx)-10, int(ly)+10), 
                                cv2.FONT_HERSHEY_DUPLEX, 1, ((0,0,0)), 2)


    if ret is not None and ret_autoAnno is not None:
        output_fn = os.path.join(labelingViz_dir, 
                     '%(stack)s_%(sec)04d_%(user)s_%(ts)s_%(user2)s_%(ts2)s_viz.jpg'% \
                                 {'stack':stack, 'sec':sec, 'user': user, 'ts': ts, 'user2':user2, 'ts2':ts2})
    elif ret is not None:
        output_fn = os.path.join(labelingViz_dir, 
                     '%(stack)s_%(sec)04d_%(user)s_%(ts)s_viz.jpg'% \
                                 {'stack':stack, 'sec':sec, 'user': user, 'ts': ts})            
    else:
        output_fn = os.path.join(labelingViz_dir, 
             '%(stack)s_%(sec)04d_%(user2)s_%(ts2)s_viz.jpg'% \
                         {'stack':stack, 'sec':sec, 'user2': user2, 'ts2': ts2})
        
    cv2.imwrite(output_fn, viz[..., ::-1])

In [ ]:
# For MICCAI paper (image, scoremap, auto annotation contours)

In [ ]:
first_bs_sec, last_bs_sec = section_range_lookup[stack]

# for sec in range(first_bs_sec, last_bs_sec+1):
for sec in [141]:

    dm.set_slice(sec)
        
    ret_autoAnno = dm.load_proposal_review_result('autoAnnotate', 'latest', 'consolidated')
    if ret_autoAnno is not None:
        user2, ts2, _, annotations_autoAnno = ret_autoAnno
        
    if ret is None and ret_autoAnno is None:
        continue
    
    print sec
    
    
    dm._load_image(versions=['rgb-jpg'])
    cropped_img = dm.image_rgb_jpg[::8, ::8]
    
    scoremaps_rootdir = '/home/yuncong/csd395/CSHL_scoremaps_lossless/'
    scoremaps_dir = os.path.join(scoremaps_rootdir, stack, '%04d'%sec)

    l = '7N'
    
    scoremap_whole = bp.unpack_ndarray_file(os.path.join(scoremaps_dir, 
                                           '%(stack)s_%(sec)04d_roi1_denseScoreMapLossless_%(label)s.bp' % \
                                           {'stack': stack, 'sec': sec, 'label': l}))
    # except:
    #     sys.stderr.write('No scoremap of %s exists\n' % (l))
    #     continue


    dataset = stack + '_' + '%04d'%sec + '_roi1'

    interpolation_xmin, interpolation_xmax, \
    interpolation_ymin, interpolation_ymax = np.loadtxt(os.path.join(scoremaps_dir, 
                                                                     '%(dataset)s_denseScoreMapLossless_%(label)s_interpBox.txt' % \
                                    {'dataset': dataset, 'label': l})).astype(np.int)

    dense_scoremap_lossless = np.zeros((dm.image_height, dm.image_width), np.float32)
    dense_scoremap_lossless[interpolation_ymin:interpolation_ymax+1,
                            interpolation_xmin:interpolation_xmax+1] = scoremap_whole

    scoremap = dense_scoremap_lossless[::8, ::8]
    scoremap_height, scoremap_width = scoremap.shape[:2]

    scoremap_layer_viz = img_as_ubyte(plt.cm.hot(scoremap)[..., :3])
    scoremap_viz = img_as_ubyte(alpha_blending(scoremap_layer_viz,
                                               cropped_img,
                                               .3, 1)[..., :3])
    

#     viz = img_as_ubyte(rescale(scoremap_viz, 1./downscale_factor))
#     viz = scoremap_viz[::downscale_factor, ::downscale_factor]
                
    if ret_autoAnno is not None and len(annotations_autoAnno) > 0:
        
        for ann in annotations_autoAnno:
            
            if ann['label'] == l:
            
                vertices = np.array(ann['vertices']) / downscale_factor
                cv2.polylines(scoremap_viz, [vertices.astype(np.int)], True, (255,0,0), 2)

    #             print ann['label']


In [ ]:
display(scoremap_viz)